# Natural Language Processing 
### CS-UH 2216 - Spring 2019
## Sentiment Analysis of 100,000 IMDB movie reviews
---


## Downloading the Imdb movie review data set

In [1]:
import warnings
warnings.filterwarnings('ignore') 

import os

# The code below will check to see if the data directory exists; if not, it will download the data.
if os.path.exists("./aclImdb") == False:
    print("Downloading the Imdb movie review data set")
    !wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
    !tar xf aclImdb_v1.tar.gz
#Shell command to show the files and directories we have under aclImdb
!ls aclImdb/*

aclImdb/imdbEr.txt  aclImdb/imdb.vocab	aclImdb/README

aclImdb/test:
labeledBow.feat  neg  pos  urls_neg.txt  urls_pos.txt

aclImdb/train:
labeledBow.feat  pos	unsupBow.feat  urls_pos.txt
neg		 unsup	urls_neg.txt   urls_unsup.txt


# 1. Analysis of data

In [2]:
#Run this cell first
print("number of positive training examples:")
pos_train = !ls aclImdb/train/pos/ | wc -l
pos_train = int(pos_train[0])
print(pos_train)
print("number of negative training examples:")
neg_train = !ls aclImdb/train/neg/ | wc -l
neg_train = int(neg_train[0])
print(neg_train)
print("number of unlabelled training examples:")
unl_train = !ls aclImdb/train/unsup/ | wc -l
unl_train = int(unl_train[0])
print(unl_train)
print("number of positive testing examples:")
pos_test = !ls aclImdb/test/pos/ | wc -l
pos_test = int(pos_test[0])
print(pos_test)
print("number of negative testing examples:")
neg_test = !ls aclImdb/test/pos/ | wc -l
neg_test = int(neg_test[0])
print(neg_test)

number of positive training examples:
12500
number of negative training examples:
12500
number of unlabelled training examples:
50000
number of positive testing examples:
12500
number of negative testing examples:
12500


#### 1.1 How many reviews are for training? 75000

In [3]:
print("total number of training reviews:" )
print(pos_train + neg_train + unl_train)

total number of training reviews:
75000


#### 1.2 How many reviews are for testing? 25000

In [4]:
print("total number of training reviews:" )
print(pos_test + neg_test)

total number of training reviews:
25000


#### 1.3 How many reviews are positive (in total in training and testing)? 25000

In [5]:
print("total positive instances in training and testing" )
print(pos_train + pos_test)

total positive instances in training and testing
25000


#### 1.4 How many reviews are negative (in total in training and testing)? 25000

In [6]:
print("total negative instances in training and testing" )
print(neg_train + neg_test)

total negative instances in training and testing
25000


#### 1.5 How many reviews are unlabelled (in total in training and testing)? 50000

In [7]:
print("total negative instances in training and testing" )
print(unl_train)

total negative instances in training and testing
50000


#### 1.6 What can we use unlabeled reviews for? 

We can use the unlabeled reviews for building unsupervised learning classification algorithms that can cluster and learn labels of reviews without being given the rating

#### 1.7 How was the positive/negative labeling done?

The positive and negative labels are classified based on the rating of the user reviews. A negative class is given to ratings <= 4 and a positive class is given to ratings >= 7 out of 10. Hence, reviews with more neural ratings are not included in the positive/negative sets

#### 1.8 Simply based on the labeling approach, do we expect some reviews to be harder than others for sentiment analysis?

Yes. This is because some reviews with the same label are closer to the extreme ends of the scale (1 ratings) while other reviews are closer to the neural end (4 rating). We would expect reviews with the same label but closer to the extreme end of the scale, to be easier to analyze.

#### 1.9 How many are the most negative review [1] (train and test)?

In [8]:
max_neg_test = !ls aclImdb/test/neg/ | grep "\w*1.txt" | wc -l
max_neg_test = int(max_neg_test[0])
max_neg_train = !ls aclImdb/train/neg/ | grep "\w*1.txt" | wc -l
max_neg_train = int(max_neg_train[0])
print("Total number of most negative reviews")
print(max_neg_test + max_neg_train)

Total number of most negative reviews
10122


#### 1.10 How many are the most positive reviews [10] (train and test)?

In [9]:
max_pos_test = !ls aclImdb/test/pos/ | grep "\w*10.txt" | wc -l
max_pos_test = int(max_pos_test[0])
max_pos_train = !ls aclImdb/train/pos/ | grep "\w*10.txt" | wc -l
max_pos_train = int(max_pos_train[0])
print("Total number of most positive reviews")
print(max_pos_test + max_pos_train)

Total number of most positive reviews
9731


## Load data to memory

In [10]:
import sklearn
from sklearn.datasets import load_files #load_files load text files with categories as subfolder names; 

# Directory of our data
traindir = r'./aclImdb/train'
testdir = r'./aclImdb/test'

# load pos/neg train and test data
train=load_files(traindir,categories=['pos','neg']) #load_files shuffles the text and categories by default.
test=load_files(testdir,categories=['pos','neg'])

# load an object with all the training data (positive, negative and unlabeled)
alltrain = load_files(traindir,categories=['pos','neg','unsup'])
#load_files return a dictionary-like object:
#1. 'data': the raw text data to learn
#2. 'target': the classification labels (integer index)
#3. 'target_names': the meaning of the labels
#4. 'filenames': the name of the file holding the data point

In [11]:
#Browse an example
#For example, if we want to see data point with index i 
i = 13374
print("Index  = %3d\n" % (i))
print("Text = %s\n" % (train.data[i]))
print("Label = %s\n" %(train.target_names[train.target[i]]))
print("Filename = %s\n" % (train.filenames[i]))

Index  = 13374

Text = b'Since this movie was based on a true story of a woman who had two children and was not very well-off, it was just scary as to how real it really was! The acting is what gave the movie that push to greatness.<br /><br />Diane Keaton portrayed the main character, Patsy McCartle who had two sons whom she adored. Her performance is what made the real life story come to life on a television screen. It was very hard to watch some of the scenes since they were so real as to what happens when one becomes addicted to drugs.<br /><br />Just watching this very loving mother go from sweet to not caring at all was hard, but so true. I have known people who have gone through withdrawl and it was very much like what happened in this movie, from what I remember.<br /><br />I also thought that it was very risky for the director to want to make a movie out of what happened to this woman. Yet it was done so well. I applaud the director for making this movie.<br /><br />I highly r

# 2. Investigating the quality of data

In [12]:
#Function to determine rating given the review filename
import re
def getRating(filename):
    match = re.search("_(.+).txt",filename)
    if match: #If we have found something
        return(int(match.group(1))) #Return the first capture group ()
    else:
        return None

In [13]:
def find_and_print_rating(rating):
    #listing filename, label, and at least the first 10 words of text
    for i in range(len(train.filenames)):
        if(getRating(train.filenames[i]) == rating):
            print("Label = %s" %(train.target_names[train.target[i]]))
            print("Filename: ", train.filenames[i])
            print("Rating: ", getRating(train.filenames[i]))
            print("Text: ", train.data[i][0:200])
            break

#### 2.1 Identify an example of a strong negative sentiment based on human ratings

In [14]:
find_and_print_rating(1)

Label = neg
Filename:  ./aclImdb/train/neg/6802_1.txt
Rating:  1
Text:  b"Words can't describe how bad this movie is. I can't explain it by writing only. You have too see it for yourself to get at grip of how horrible a movie really can be. Not that I recommend you to do th"


#### 2.2 Identify an example of a weak negative sentiment based on human ratings

In [15]:
find_and_print_rating(4)

Label = neg
Filename:  ./aclImdb/train/neg/9503_4.txt
Rating:  4
Text:  b'Also known in a different form as "House of Exorcism," this messy<br /><br />little film takes itself so seriously as to kill any entertainment value<br /><br />whatsoever.<br /><br />The spare plot i'


#### 2.3 Identify an example of a strong positive sentiment based on human ratings

In [16]:
find_and_print_rating(10)

Label = pos
Filename:  ./aclImdb/train/pos/11485_10.txt
Rating:  10
Text:  b'Zero Day leads you to think, even re-think why two boys/young men would do what they did - commit mutual suicide via slaughtering their classmates. It captures what must be beyond a bizarre mode of be'


#### 2.4 Identify an example of a weak positive sentiment based on human ratings 

In [17]:
find_and_print_rating(7)

Label = pos
Filename:  ./aclImdb/train/pos/9846_7.txt
Rating:  7
Text:  b'Ok, at the beginning it looked like "Shrek" - the loner that is persistently followed by the comic relief. Then it evolves into something really compelling, as the gauntlet is set. And the result is a'


#### 2.5 List one observation of a feature of the text that you think will be helpful to predict sentiment.

It seems that the choice of vocabulary seems to be correlated with the rating of the review. Certain vocabulary such as "Horrible" are seen significantly more frequently in negative instances than in positive instances

In [18]:
#Number of reviews with "Horrible" in contents.
negativeOccurances = 0
positiveOccurances = 0
for i in range(len(train.filenames)):
    if(train.target_names[train.target[i]] == 'pos'):
        if 'horrible' in str(train.data[i]):
            positiveOccurances += 1
    else:
        if 'horrible' in str(train.data[i]):
            negativeOccurances += 1
print("Number of times \"horrible\" appeared in negative reviews: ", negativeOccurances)
print("Number of times \"horrible\" appeared in positive reviews: ", positiveOccurances)

Number of times "horrible" appeared in negative reviews:  796
Number of times "horrible" appeared in positive reviews:  150


# 3. Building a basic sentiment analyzer

Vectorize our text data

In [19]:
from sklearn.feature_extraction.text import CountVectorizer #Vectorize our text with top 1000 frequent vocabulary

In [20]:
#Create the vectorizer and transform our data
vectorizer = CountVectorizer(max_features=1000,analyzer='word',lowercase=True) #Tokenize at word level
vectorizer.fit(alltrain.data)
train_data_vectorized = vectorizer.transform(train.data)

Implement a logistic linear regression model

In [21]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='liblinear')
model.fit(train_data_vectorized, train.target)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)

Predict with the model

In [41]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
test_pred = model.predict(vectorizer.transform(test.data))
#Preliminary evaluation with sklearn
print ("Accuracy  = %3.2f%%" % (100*accuracy_score(test.target, test_pred)))
print ("Average Precision = %3.2f%%" % (100*precision_score(test.target, test_pred, average='macro')))
print ("Average Recall    = %3.2f%%" % (100*recall_score(test.target, test_pred, average='macro')))
print ("Average F1-score  = %3.2f%%" % (100*f1_score(test.target, test_pred, average='macro')))
# The scores for accuracy, and average precision, recall and F1-score are similar. 
# This is in part an effect of the balanced nature of the data.

Accuracy  = 85.96%
Average Precision = 85.97%
Average Recall    = 85.96%
Average F1-score  = 85.95%


#### 3.1 Evaluating 

In [53]:

num_neg = 0
num_pos = 0
neg_correct = 0
neg_incorrect = 0
pos_correct = 0
pos_incorrect = 0

for i in range(len(test.target)):
    if test.target[i] == 0:
        num_neg +=1
        if test_pred[i] == 0:
            neg_correct += 1
        elif test_pred[i] == 1:
            neg_incorrect += 1
    elif test.target[i] == 1:
        num_pos += 1
        if test_pred[i] == 0:
            pos_incorrect += 1
        elif test_pred[i] == 1:
            pos_correct += 1

print("Total instances %d" % (num_neg + num_pos))
print("Total correct predictions %d" % (pos_correct + neg_correct) )
print("Correctly predicted negative %d" % (neg_correct))
print("Incorrectly predicted negative %d" % (neg_incorrect))
print("Correctly predicted positive %d" % (pos_correct))
print("Incorrectly predicted positive %d" % (pos_incorrect))

Total instances 25000
Total correct predictions 21489
Correctly predicted negative 10617
Incorrectly predicted negative 1883
Correctly predicted positive 10872
Incorrectly predicted positive 1628


In [94]:
#Accuracy = Correctly predicted / Total
Accuracy = (pos_correct + neg_correct)/(num_neg + num_pos)
print("Accuracy: %2.2f%%" % (100*Accuracy))
print()

#Neg label precision = neg_correct / (neg_correct + pos_incorrect)
neg_precision = neg_correct / (neg_correct + pos_incorrect)
print("Neg label precision: %2.2f%%" % (100*neg_precision))

#Recall of Neg = neg_correct / num_neg
neg_recall = neg_correct / num_neg
print("Neg label recall: %2.2f%%" % (100*neg_recall))

#F1-score harmonic mean of precision and recall. = 
neg_f1 = 1/( ((1/neg_precision)+(1/neg_recall))/2 )
print("Neg F1 score: %2.2f%%" % (100*neg_f1))
print()

#Pos label precision = pos_correct / (pos_correct + neg_incorrect)
pos_precision = pos_correct / (pos_correct + neg_incorrect)
print("Pos label precision: %2.2f%%" % (100*pos_precision))

#Recall of Pos = pos_correct / num_pos
pos_recall = pos_correct / num_pos
print("Pos label recall: %2.2f%%" % (100*pos_recall))

#F1-score harmonic mean of precision and recall. = 
pos_f1 = 1/( ((1/pos_precision)+(1/pos_recall))/2 )
print("Pos F1 score: %2.2f%%" % (100*pos_f1))
print()

print("Average precision %2.2f" % (100*(neg_precision + pos_precision)/2))
print("Average recall %2.2f" % (100*(neg_recall + pos_recall)/2))
print("Average F1 score %2.2f%%" % (100*(neg_f1 + pos_f1)/2))

Accuracy: 85.96%

Neg label precision: 86.70%
Neg label recall: 84.94%
Neg F1 score: 85.81%

Pos label precision: 85.24%
Pos label recall: 86.98%
Pos F1 score: 86.10%

Average precision 85.97
Average recall 85.96
Average F1 score 85.95%


## 